# Example downloader of the Sentinel 2 L2A data from AWS

This notebook goes through the process to identifying and downloading the Sentinel 2 L2A data from Amazon AWS.  
It is closely based on [Martin D. Maas](https://github.com/mdmaas) instructions which can be found [here](https://www.matecdev.com/posts/landsat-sentinel-aws-s3-python.html).

In [ ]:
# Update sys path so notebook can access the agromanagement package
import sys
sys.path.append('../')

## Step 1: configure access to the AWS servers
The first step consists in the creation and configuration of an AWS account. This can be done following the instructions found on the [AWS website](https://repost.aws/knowledge-center/create-and-activate-aws-account).  
Next, we need to install and set up the [AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html).

## Step 2: configure access to the STAC server containing the search catalog of Sentinel-2
We now need to define the STAC server and explore the available collections. 
For Sentinel 2, we will be using the [following server](https://earth-search.aws.element84.com/v0), maintained by the team of [Element84](https://www.element84.com). 
The python package sat-search, which is included in the Conda environment for this project, is the client that will allow us to search and explore the Sentinel 2 data collections.

In [ ]:
import satsearch

# Declare server
SENTINEL_SERVER = "https://earth-search.aws.element84.com/v0"

# build stac collection
sentinel_stac = satsearch.Search.search(url=SENTINEL_SERVER)
print(f"Found {str(sentinel_stac.found())} items")

## Step 3: Define a location of interest
We query the Sentinel 2 L2A dataset spatially from the geometry of a parcel or set of coordinates of interest.  
In this example we are going to use a parcel with its geometry (i.e., a shapefile).

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Load a parcel from data
parcels = gpd.read_file("../resources/" + "lcm2021_tile_11_1014_647.geojson")
parcel = parcels.iloc[1025:1026,:]
print(parcel)

# plot the parcel
parcel.plot()
plt.show()

In [ ]:
from shapely.geometry import mapping

# define bounding box of the parcel of interest
geometry = parcel.iloc[0,:]["geometry"]
geom_json = mapping(geometry)

## Step 4: define the temporal timeframe of interest

In [ ]:
# define a time range of interest
time_range = '2019-01-01/2021-12-31'

# Step 4: explore the collection of available data for the location of interest and the specified temporal interval

In [ ]:
sentinel_search = satsearch.Search.search( 
    url = SENTINEL_SERVER,
    intersects = geom_json,
    datetime = time_range,
    collections = ['sentinel-s2-l2a-cogs']
)

sentinel_items = sentinel_search.items()
print(sentinel_items.summary())

We are going to focus, for this example, on only one of the items that are available for the selected location and timeframe

In [ ]:
item = sentinel_items[0]
print(item)

We can look at the content of the object `item`, querying the `assets` property:

In [ ]:
print(item.assets.keys())

We can see that our `item` contains some information (*overview, thumbnail, metadata, visual, info*) and a series of spectral bands (*B01* to *B12*, and *AOT, B8A, WVP, SCL*).  
Information on the bands can be found [here](https://docs.sentinel-hub.com/api/latest/data/sentinel-2-l2a/).

**NOTE:**  
Different bands have different spatial resolutions, from 10m to 160m.

We can check the content of a band as follows:

In [ ]:
print(item.assets["B02"].keys())

We can see that `B02` is the blue band:

In [ ]:
print(item.assets["B02"]['eo:bands'])

## STEP 5: Download the data
Now we can proceed to download the data of interest. This is done using the function `download_sentinel` which takes the following arguments:
- `geotiff_file`: the link to the data to be donwloaded
- `output_folder`: where the downloaded data will be stored.
The stored data will be contained in `ouput_folder` into a series of subfolders, one for each tile at each point in time, follwong standard Sentinel 2 naming convention: 
***MMM_XXXXX_YYYYMMDD_A_L2A*** where:
- ***MMM*** is the mission ID(S2A/S2B)
- ***XXXXX*** is the tile number field (e.g. 30UWF where *30* is the UTM zone, *U* is the latitude band and *WF* is the grid square)
- ***YYYYMMDDHHMMSS*** represents the capture date.

**NOTE:**  
more info can be retrieved from the `info` key in the `item.assets` dictionary

In [ ]:
from agromanagement.core.utils import download_sentinel

SENTINEL_OUTPUT_FOLDER = "../resources/sentinel_l2a/"
geotiff_file = item.assets["B02"]["href"]

tile_data = download_sentinel(
    geotiff_file=geotiff_file,
)
print(tile_data.keys())
print(tile_data["metadata"])
